# Molecular Dynamics using Pytorch Autograd


This notebook will run a 2D Lennard Jones fluid simulation where the forces are calculated from the potential energy function using PyTorch Autograd.


## System settings

First we define the system settings and import the modules needed.

In [1]:
import numpy as np
import torch
from ase import Atoms, io

# Settings
M = 10 # initial coordinate grid spacing
N = M**2 # Num atoms
L = 20.0 # Box length
dt = 0.01 # timestep
kT = 1.0 # KbT
gamma = 1.0 # Friction
sigma = 1.0 # LJ sigma
epsilon = 2.0 #  LJ epsilon
Nsteps = 10000 # Number of timesteps
Nout = 100 # Frequency of output


## Energy function

We now define the energy function of the system. This takes a tensor of the atomic positions which has `size=(Natoms, 2)` and returns a tensor of `size=(1)` which contains the total potential energy of the system.

It is important that all the math operations are `torch` tensor operations.

In [2]:
def energy(positions):
    "Compute the energy of a Lennard-Jones system."
   
    # get [i,j] pairs   
    a, b = torch.triu_indices(N, N, 1)
    
    # compute the displacement vectors
    d = positions[a] - positions[b]
    
    # apply PBCs
    d = d - torch.round(d/L)*L

    # compute distance^2
    r2 = torch.sum(d**2, dim=1)
    
    c6 = (sigma**2 / r2)**3
    c12 = c6**2
    
    return torch.sum(4 * epsilon * (c12 - c6))

## Run simulation

We can now run the simulation. We use an initial configuration that is a grid.

In [3]:
# Simulation setup

# for clarity we will keep all variables as numpy arrays except from the coordinates we put into the energy function

# make a grid
line = torch.linspace(0,L-2.0,M)

# coordinates
x = torch.cartesian_prod(line,line).numpy()

# velocities
v = np.zeros((N,2))

# Main simulation loop
frames=[]
for t in range(Nsteps):
    
    # create x as a torch.Tensor with equires_grad=True 
    # this tells autograd to track operations on x so it can compute derivatives using the chain rule
    x = torch.tensor(x, requires_grad=True)

    # compute energy
    E = energy(x)

    # compute forces using Autograd

    # run a backward pass on E.
    E.backward()

    # this tells Autograd to compute the partial derivative of E wrt to its inputs.
    # the values of ∂E /∂x are stored in the .grad attribute of x
    # we can now get the forces by reading the grad
    # F = - ∂E /∂x
    F = -x.grad 

    # convert back to numpy arrays
    F=F.detach().numpy()
    x=x.detach().numpy()
 
    # Integrate in NVT using leap-frog Verlet Langevin
    alpha = np.exp(-gamma*dt)
    v = v*alpha + F*(1.0-alpha)/gamma + np.sqrt(kT*(1-alpha**2))*np.random.normal(size=v.shape)
    x = x + v*dt
    
    # apply PBCs
    x = x - np.floor(x/L)*L
    
    # Record output
    if t%Nout==0:
        print(t,"/", Nsteps)
        xyz = np.hstack((x, np.zeros((N,1))))
        frame = Atoms(''.join(['C']*N), positions = xyz, cell=(L,L,L), pbc=True)
        frames.append(frame)

# write trajectory using ASE
io.write("lj_autograd_traj_2d.xyz", frames)

0 / 10000
100 / 10000
200 / 10000
300 / 10000
400 / 10000
500 / 10000
600 / 10000
700 / 10000
800 / 10000
900 / 10000
1000 / 10000
1100 / 10000
1200 / 10000
1300 / 10000
1400 / 10000
1500 / 10000
1600 / 10000
1700 / 10000
1800 / 10000
1900 / 10000
2000 / 10000
2100 / 10000
2200 / 10000
2300 / 10000
2400 / 10000
2500 / 10000
2600 / 10000
2700 / 10000
2800 / 10000
2900 / 10000
3000 / 10000
3100 / 10000
3200 / 10000
3300 / 10000
3400 / 10000
3500 / 10000
3600 / 10000
3700 / 10000
3800 / 10000
3900 / 10000
4000 / 10000
4100 / 10000
4200 / 10000
4300 / 10000
4400 / 10000
4500 / 10000
4600 / 10000
4700 / 10000
4800 / 10000
4900 / 10000
5000 / 10000
5100 / 10000
5200 / 10000
5300 / 10000
5400 / 10000
5500 / 10000
5600 / 10000
5700 / 10000
5800 / 10000
5900 / 10000
6000 / 10000
6100 / 10000
6200 / 10000
6300 / 10000
6400 / 10000
6500 / 10000
6600 / 10000
6700 / 10000
6800 / 10000
6900 / 10000
7000 / 10000
7100 / 10000
7200 / 10000
7300 / 10000
7400 / 10000
7500 / 10000
7600 / 10000
7700 / 1000

Here is an example trajectory that it creates

![display image](./traj_2d.gif)